# 스타벅스 데이터

In [5]:
# Starbucks data

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import chromedriver_autoinstaller
import pandas as pd
import time
import re

# chrome driver 생성
chromedriver_autoinstaller.install()

'/Users/daniel_choi/opt/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/108/chromedriver'

In [8]:
url = 'https://www.starbucks.co.kr/store/store_map.do'
# webdriver.Chrome의 인자로 들어갈 값은 컴퓨터마다 달라질 수 있다. 가장 처음의 셀을 실행했을때
# chrome driver 생성을 통해서 비롯되는 out값을 인자에 입력해야한다.
driver = webdriver.Chrome('/Users/daniel_choi/opt/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/108/chromedriver')
driver.get(url)

/var/folders/5g/z1vg08pd2_97rjb0w3yzhl4m0000gn/T/ipykernel_19161/1895378607.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/daniel_choi/opt/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/108/chromedriver')


In [7]:
# 비는 부분이 없도록 최대화
driver.maximize_window()

In [ ]:
# 매장찾기 - 지역 검색 탭 클릭
driver.find_elements(By.CSS_SELECTOR, 'header.loca_search > h3 > a')[0].click()

In [ ]:
# 지역 검색 중 "서울" 선택

# 확인 코드
# driver.find_elements(By.CSS_SELECTOR, 'ul.sido_arae_box > li')[0].text
# '서울'
driver.find_elements(By.CSS_SELECTOR, 'ul.sido_arae_box > li')[0].click()

In [ ]:
# 서울 에서 "전체"클릭

# 확인코드
# driver.find_elements(By.CSS_SELECTOR, 'ul.gugun_arae_box > li')[0].text
# '전체'
driver.find_elements(By.CSS_SELECTOR, 'ul.gugun_arae_box > li')[0].click()

In [ ]:
# 서울 전체 검색결과가 22.07.20 기준으로 585개다.

# 확인 1 (갯수)
# len(driver.find_elements(By.CSS_SELECTOR, 'ul.quickSearchResultBoxSidoGugun > li'))
# 585

# 확인 2 (처음 text)
# driver.find_elements(By.CSS_SELECTOR, 'ul.quickSearchResultBoxSidoGugun > li')[0].text
# '역삼아레나빌딩\n서울특별시 강남구 언주로 425 (역삼동)\n1522-3232\n리저브 매장 2번'

# 확인 3 (마지막 text)
# driver.find_elements(By.CSS_SELECTOR, 'ul.quickSearchResultBoxSidoGugun > li')[-1].text
# '' 라고 나온다. 뭔가 잘못되었다.

# 확인 4 (마지막 text)를 체크했을 때, 확인 3이 안먹어서
# driver.find_elements(By.CSS_SELECTOR, 'ul.quickSearchResultBoxSidoGugun > li')[184].text
# 페이지에서 보이질 않기때문에 긁어올 수 없는것일까? 184번 인덱스 (185번째)의 것도 ''로 표기가 된다.

# 확인 5
# driver는 보이는 페이지에 국한해서 나올 수 있기 때문에 Beautifulsoup로 시도해주자.
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
len(soup.select('ul.quickSearchResultBoxSidoGugun > li'))
# 585
main = soup.select('ul.quickSearchResultBoxSidoGugun > li')

In [ ]:
main[-1]
'''
<li class="quickResultLstCon" data-code="3801" data-hlytag="null" data-index="584" data-lat="37.60170912407773" data-long="127.07841136432036" data-name="중화역" data-storecd="1749" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="중화역" data-store="1749" data-yn="N">중화역  </strong> <p class="result_details">서울특별시 중랑구 봉화산로 35 <br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>
'''

# ok 이걸로 진행하자. 0번 인덱스의 값도 이상없다.

In [ ]:
# 정규표현식으로 내가 원하는 값을 제거해보려 했지만, 내가 원하는 값은 아니였다.
# 즉, 코드는 여기선 별반 도움이 안되었다.

def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', inputString)
  return text_rmv

In [ ]:
# 자료 긁어오기
# 확인 1
# 매장 명
store = i('strong')[0].text.strip()
# 매장 주소
# 반복문 버전에 성공했지만, 
# 다음엔 연산을 줄이기 위해 다른 방법을 찾자.
for i in main[0]('p')[0].text.split()[:-1]:
    print(i, end=' ')

# 매장 구
gu = i('p')[0].text.split()[1]
# 매장 전화 번호
# 현재로써는 전화번호는 데이터상 중요하지 않다.
# 하지만 반드시 얻어내자.
# main[0]('p')[0].text.split()[-1:][0]
# '(역삼동)1522-3232'

# 매장 위도 (lat)
lat = i['data-lat']
# 매장 경도 (lng)
lng = i['data-long']

In [ ]:
datas = []
for i in tqdm(main):

    # 자료 긁어오기
    # 확인 1
    # 매장 명
    name = i('strong')[0].text.strip()
    # 매장 주소
    # 반복문 버전에 성공했지만, 
    # 다음엔 연산을 줄이기 위해 다른 방법을 찾자.


    # address = [a for a in i('p')[0].text.split()[:-1]]
    address = i('p')[0].text.replace('1522-3232', '')

    gu = i('p')[0].text.split()[1]
    # 매장 전화 번호
    # 현재로써는 전화번호는 데이터상 중요하지 않다.
    # 하지만 반드시 얻어내자.
    # main[0]('p')[0].text.split()[-1:][0]
    # '(역삼동)1522-3232'

    # 매장 위도 (lat)
    lat = i['data-lat']
    # 매장 경도 (lng)
    lng = i['data-long']

    # pandas DataFrame을 위해서 이렇게 미리 지정하여 만들어 준다.
    data = {
        '매장이름':name,
        '주소':address,
        '구':gu,
        '위도':lat,
        '경도':lng,
        '브랜드':'스타벅스'
    }
    datas.append(data)
    
print(datas)

In [ ]:
# 주소 테스트 코드
soup.select('div#mCSB_3_container > ul > li')[0]('p')[0].text[:-15]
# '서울특별시 강남구 언주로 425'
# 이게 올바르지 않다고 생각한다. 근본적으로 주소에 변동성이 많으면 이 코드는 무쓸모다.
# 다음엔 좀 더 나은 방법을 찾아내자.

In [ ]:
# 정리된 마무리 데이터를 엑셀화 시켜주기
df_starbucks = pd.DataFrame(datas)
df_starbucks.to_excel('starbucks_data.xlsx')

In [ ]:
# 스타벅스로 볼일 다 봤으니 꺼주자.

driver.quit()

---

# 이디야 데이터

In [ ]:
# Ediya Data

# import 는 starbucks때 다 해왔으니깐 생략하고 진행 ㄱㄱ

new_url = 'https://www.ediya.com/contents/find_store.html'
# 이 변수가 이미 선언 되어있다 하더라도 같은 셀에 없으면 ConnectionRefusedError 뜨니깐 꼭 같이!
driver = webdriver.Chrome('/Users/daniel_choi/miniforge3/envs/ds_study/lib/python3.10/site-packages/chromedriver_autoinstaller/103/chromedriver')
driver.get(new_url)

In [ ]:
# 화면 최대화
driver.maximize_window()

In [ ]:
# '주소'클릭
driver.find_elements(By.CSS_SELECTOR,'#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a')[0].click()

In [ ]:
# 검색을 위한 구 리스트 만들기
# 스타벅스 만들때 활용한 데이터 이용

gu_list = df_starbucks.구.unique()

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# 이디야 지역 스크래핑을 위한 메인 변수 설정
main = soup.select('ul#placesList >li.item')

In [ ]:
# 각 구별로 매장 수가 다를테니, main을 재설정 해주는 것을 생각해보자.

# 매장 명
# main[0]('dt')[0].text

# 매장 주소
main[0]('dd')[0].text[:-6]
# '서울 중랑구 망우로 460'

In [ ]:
# 주소 값에다가 서울 각 구 이름 넣어서 빼주기

# 테스트 1 (값 보내기)
driver.find_elements(By.CSS_SELECTOR, '#keyword')[0].send_keys('강남구')

# 테스트 2 (기존 값 지우기)
driver.find_elements(By.CSS_SELECTOR, '#keyword')[0].clear()

# 테스트 3 (엔터키 누르기)
driver.find_elements(By.CSS_SELECTOR, '#keyword')[0].send_keys(Keys.ENTER)

In [ ]:
# 위도, 경도를 뽑아내고 싶었지만, 있는것도 있고 없는것도 있고 빈도가 듬성듬성해서 googlemap을 이용하기로 결정함
main[1]('a')[0]['onclick']
# "panLatTo('127.094182772191','37.5896269575279','1');fnMove();"

In [ ]:
# 반복문 테스트 (위도, 경도는 추후 넣기위해 나머지 3개)

ediya_datas = []
for i in tqdm(gu_list):
        
    # 기존 입력코드, 시도 실패.
    # UnexpectedAlertPresentException: Alert Text: 검색 결과가 너무 많습니다. 범위를 좁혀 주시기 바랍니다.
    # 라는 에러가 떴다. 
    # 따라서 대기시간, 입력시간에 문제가 있다고 판단하고, 이를 보완하기 위해서 코드를 변경해줬다.

    '''
    # (기존 값 지우기)
    driver.find_elements(By.CSS_SELECTOR, '#keyword')[0].clear()
    # (값 보내기)
    driver.find_elements(By.CSS_SELECTOR, '#keyword')[0].send_keys(i)
    # (엔터키 누르기)
    driver.find_elements(By.CSS_SELECTOR, '#keyword')[0].send_keys(Keys.ENTER)
    
    time.sleep(2)
    '''
    
    # 2차시도 기본
    keyword_css = '#keyword'
    
    # 기존 값 지우기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, keyword_css))).clear()
    
    # 값 보내기
    # 이게 이번의 핵심 코드였다. ****
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, keyword_css))).send_keys(f'서울 {i}')
    
    # 엔터키 누르기
    search_css = '#keyword_div > form > button'
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, search_css))).click()
        
    
    # 각 구마다 매장의 수가 다를테니, 재설정을 해준다.
    main = soup.select('ul#placesList >li.item')
    for a in main:
        
        # 매장 명
        name = a('dt')[0].text

        # 매장 주소
        address = a('dd')[0].text
        
        # 구
        gu = i
        
        data = {
            '매장이름':name,
            '주소':address,
            '구':gu,
            '브랜드':'이디야'
        }
        
        ediya_datas.append(data)

# 계속 해서 UnexpectedAlertPresentException: Alert Text: 검색 결과가 너무 많습니다. 범위를 좁혀 주시기 바랍니다.
# 에러가 떴다. 구글링을 해봐도, 알럿을 무시하는 코드만 나오지, 그것은 내가 원하는것이 아니였다.
# 값에 대해서 누락 없이 가져가고 싶으니깐, 그래서 유튜브 자료로 찾아서 참조했다.
# 참조: https://www.youtube.com/watch?v=qobe7k1CmGc&t=1099s

# 그외 참조는 selenium 공식 문서를 기반으로 만들어 지긴 했는데,
# 유튜브에서 딱 하나지만 '강서구' 를 예로 들었을때, 부산에도 강서구, 서울에도 강서구가 있기 때문에 결과가 많이 나오는 것이였다.
# 그래도 그렇지 그게 많으면 얼마나 많다고 그거에 출력을 제한하냐;

In [ ]:
# len(ediya_datas)
# 675

ediya_datas[0]
# {'매장이름': '금란망우점', '주소': '서울 중랑구 망우로 460', '구': '강남구'}

In [ ]:
df_ediya = pd.DataFrame(ediya_datas)
df_ediya.tail()

---

In [ ]:
# 이디야 위도, 경도 따주기

In [ ]:
# 이디야 위도, 경도 따주기

# 먼저 위도, 경도 컬럼 생성해주기

import numpy as np
df_ediya['위도'] = np.nan
df_ediya['경도'] = np.nan

In [ ]:
# 이디야 위도, 경도 따주기

import googlemaps
google_key = '본인의 구글API 값 사용하세염'
gmaps = googlemaps.Client(key=google_key)
gmaps

In [ ]:
# 이 자료가 위도, 경도따기의 척도다.
# 이 자료를 토대로 보게되면 위도 경도를 딸 수 있는 루트는 3가지가 있다.
# key값으로는 geometry - location,
# viewport - northeast,
# southwest
# 하지만 이중에서 가장 '짧게'나온 위도 경도는 geometry와 location으로 이루어진 것으로써, 이것을 사용하도록 한다.

for idx, rows in df_ediya.iterrows():
    # rows.주소
    print(gmaps.geocode(rows.주소, language = 'ko'))
    break
    
'''
[{'address_components': [{'long_name': '４６０', 'short_name': '４６０', 'types': 
['premise']}, {'long_name': '망우로', 'short_name': '망우로', 'types': 
['political', 'sublocality', 'sublocality_level_4']}, {'long_name': '중랑구', 'short_name': 
'중랑구', 'types': ['political', 'sublocality', 'sublocality_level_1']}, 
{'long_name': '서울특별시', 'short_name': '서울특별시', 'types': ['administrative_area_level_1', 'political']}, 
{'long_name': '대한민국', 'short_name': 'KR', 'types': ['country', 'political']}, 
{'long_name': '131-230', 'short_name': '131-230', 'types': ['postal_code']}], 
'formatted_address': '대한민국 서울특별시 중랑구 망우로 460', 'geometry': {'location': 
{'lat': 37.6000681, 'lng': 127.1031213}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': 
{'lat': 37.6014170802915, 'lng': 127.1044702802915}, 'southwest': {'lat': 37.59871911970851, 
'lng': 127.1017723197085}}}, 'place_id': 'ChIJ6U1Y60G6fDURcbe4fw-yeh8', 'plus_code': 
{'compound_code': 'J423+26 대한민국 서울특별시', 'global_code': '8Q99J423+26'}, 'types': ['street_address']}]
'''

In [ ]:
# *** 위도 경도 딸때 등, 응용할때 정말 중요한 부분이다.

for idx, rows in tqdm(df_ediya.iterrows()):
    # rows.주소
    # print(gmaps.geocode(rows.주소, language = 'ko'))
    # break
    
    lat_lng = gmaps.geocode(rows.주소, language = 'ko')
    # lat, lng을 포함하고 있는 딕셔너리의 key 값이 geometry니깐 이걸 써준다.
    lat = lat_lng[0].get('geometry')['location']['lat']
    # 37.6000681
    lng = lat_lng[0].get('geometry')['location']['lng']
    # 127.1031213
    df_ediya.loc[idx, '위도'] = lat
    df_ediya.loc[idx, '경도'] = lng

In [ ]:
df_ediya

'''

	매장이름	주소	구	브랜드	위도	경도
0	금란망우점	서울 중랑구 망우로 460 (망우동)	강남구	이디야	37.600068	127.103121
1	동원사거리점	서울 중랑구 겸재로 240 (면목동, 행복오피스텔)	강남구	이디야	37.589608	127.094165
2	망우동점	서울 중랑구 망우로 416 (망우동)	강남구	이디야	37.599128	127.098336
3	망우중앙점	서울 중랑구 용마산로115길 109 (망우동, 한일써너스빌리젠시2단지)	강남구	이디야	37.597644	127.094346
4	먹골역점	서울 중랑구 동일로157길 13 (묵동)	강남구	이디야	37.609701	127.076882
...	...	...	...	...	...	...
670	중랑교차로점	서울 중랑구 동일로 683 (면목동)	중랑구	이디야	37.591397	127.079841
671	중랑역점	서울 중랑구 망우로 198 (상봉동)	중랑구	이디야	37.593212	127.074866
672	중화동점	서울 중랑구 동일로129길 1 (중화동)	중랑구	이디야	37.599293	127.078348
673	중화역점	서울 중랑구 동일로 815, 1층	중랑구	이디야	37.603092	127.078876
674	화랑대역점	서울 중랑구 신내로25가길 2 (묵동, 현동학당)	중랑구	이디야	37.619451	127.084160
675 rows × 6 columns
'''

In [ ]:
# 정리된 마무리 데이터를 엑셀화 시켜주기
df_ediya.to_excel('ediya.xlsx')